<a href="https://colab.research.google.com/github/TaiseiYamana/optuna_study/blob/main/optuna_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optuna チュートリアル1

## 参考本
https://www.ohmsha.co.jp/book/9784274230103/
## GitHub
https://github.com/pfnet-research/optuna-book

In [1]:
# Optunaのインストール
!pip3 install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.5 MB/s eta 0:00:00


# 単目的最適化

Optuna を使って以下の数式$f$の最適化をしましょう。
<br />
<br />
$f(x,y) = (1.5 - x + xy)^2 + (2.25 -x +xy^2)^2 + (2.625 -x + xy^3)^2 $

$(-4.5\le x \le, -4.5 \le y \le 4.5)$

$f$を最小化する最適解 : $( x, y) = (3,0.5)$
<br />
<br />

Optunaでは以下の 4 つのステップを実装していきます。
1.   目的関数を定義する
2.   目的関数の内部で適当な変数を決める ***(suggest)***
3.   実験 ***study*** オブジェクトを作成する。
4.   施行 ***(trial)*** の回数を設定して最適化を開始する ***(optimize)***:

In [2]:
# Optunaをインポート
import optuna

# 目的関数を定義
def objective(trial): # 引数に　trial を指定
    # サジェストAPIで目的関数への入力を定案
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)

    # 目的関数の評価値をreturn
    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2

# 最適化を実行
study = optuna.create_study(direction="minimize") # directionで最小化を指定
study.optimize(objective, n_trials=1000)

# 最適化の結果を確認
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

[I 2023-07-03 16:29:17,614] A new study created in memory with name: no-name-7c638356-731a-437a-b716-35d324ca38e4
[I 2023-07-03 16:29:17,620] Trial 0 finished with value: 31431.071696427676 and parameters: {'x': -2.257818779483909, 'y': 4.289169909995401}. Best is trial 0 with value: 31431.071696427676.
[I 2023-07-03 16:29:17,623] Trial 1 finished with value: 66164.04903174767 and parameters: {'x': -4.39928156823605, 'y': -3.8071043289458157}. Best is trial 0 with value: 31431.071696427676.
[I 2023-07-03 16:29:17,626] Trial 2 finished with value: 6.924983345724954 and parameters: {'x': 2.6658237989687192, 'y': 0.872105835083901}. Best is trial 2 with value: 6.924983345724954.
[I 2023-07-03 16:29:17,629] Trial 3 finished with value: 2.027690023424789 and parameters: {'x': 3.0580348356990212, 'y': 0.7462784460516616}. Best is trial 3 with value: 2.027690023424789.
[I 2023-07-03 16:29:17,633] Trial 4 finished with value: 319.20876163328103 and parameters: {'x': 3.941955391476247, 'y': -1.

Best objective value: 2.2273895529870733e-05
Best parameter: {'x': 3.0118624020552613, 'y': 0.502970767802308}


# 多目的最適化
多目的最適化は最適化する評価値が2つ以上ある最適化です。  
例として以下の数式$f_1, f_2$の最適化をしましょう。
<br />
<br />
$f_1(x,y) = 4x^2+4y~2$

$f_2(x,y)=(x-5)^2 + (y -5)^2$

$(0 \le x \le 5, 0 \le y \le 3)$
<br />
<br />
$f_1$を最小化、$f_2$を最大化する$(x,y)$の探索


In [3]:
# Optunaをインポート
import optuna


# 目的関数を定義
def f1(x, y):
    return 4 * x**2 + 4 * y**2

def f2(x, y):
    return (x - 5)**2 + (y - 5)**2

def objective(trial): # 引数に　trial を指定
    # サジェストAPIで目的関数への入力を定案
    x = trial.suggest_float("x", 0, 5)
    y = trial.suggest_float("y", 0, 3)

    v1 = f1(x, y)
    v2 = f2(x, y)

    # 変更点1: 目的関数が複数の値を返す
    return v1, v2

# 最適化を実行
study = optuna.create_study(
    # 変更点2: 目的ごとに最適化の方向を指定
    directions=["minimize", "minimize"]
)
study.optimize(objective, n_trials=100)

# 最適化の結果を確認
print("[Best Trials]")
# 変更点3: Study.best_trialのかわりにStudy.best_trialsを使用
for trial in study.best_trials:
    # 変更点4: FrozenTrial.valueのかわりにFrozenTrial.valuesを使用
    print(f"- [{trial.number}] params={trial.params}, values={trial.values}")

[I 2023-07-03 16:29:48,725] A new study created in memory with name: no-name-c348a4b1-b657-4888-816e-52a650cc1fa2
[I 2023-07-03 16:29:48,729] Trial 0 finished with values: [56.910028978994006, 12.8518916972655] and parameters: {'x': 3.286498646851641, 'y': 1.851062907896659}. 
[I 2023-07-03 16:29:48,732] Trial 1 finished with values: [103.1219214410581, 11.201071172702843] and parameters: {'x': 4.798680608608173, 'y': 1.6592603101479955}. 
[I 2023-07-03 16:29:48,736] Trial 2 finished with values: [64.29457587404102, 9.385938438614339] and parameters: {'x': 2.8899018383898416, 'y': 2.7788687145997497}. 
[I 2023-07-03 16:29:48,739] Trial 3 finished with values: [42.37649234456057, 16.11967231402839] and parameters: {'x': 1.630325863212481, 'y': 2.8171192139986942}. 
[I 2023-07-03 16:29:48,742] Trial 4 finished with values: [104.8850252779847, 10.95367290590775] and parameters: {'x': 4.832131715896091, 'y': 1.6946266254627513}. 
[I 2023-07-03 16:29:48,744] Trial 5 finished with values: [4

[Best Trials]
- [2] params={'x': 2.8899018383898416, 'y': 2.7788687145997497}, values=[64.29457587404102, 9.385938438614339]
- [6] params={'x': 2.2052780712967737, 'y': 2.5742804029858313}, values=[45.96068385975725, 13.694586222113262]
- [8] params={'x': 0.7011110210159155, 'y': 0.17939568583721488}, values=[2.094957903547937, 41.71867240735568]
- [9] params={'x': 1.4728968810577237, 'y': 1.2617782339626984}, values=[15.046038135726384, 26.414758383727374]
- [15] params={'x': 3.881029567127477, 'y': 2.920068091224631}, values=[94.3567526332238, 5.578211574784872]
- [23] params={'x': 1.757528172478144, 'y': 1.741187490416007}, values=[24.482556615342226, 21.13348252489405]
- [24] params={'x': 3.141415752274479, 'y': 2.594552832066104}, values=[66.4007893080819, 9.24051148361464]
- [25] params={'x': 0.613936921093986, 'y': 0.5464901616952635}, values=[2.702280159648314, 39.07129921201958]
- [26] params={'x': 2.05598341765839, 'y': 2.098123523883753}, values=[34.51676054064261, 17.088120